<a href="https://colab.research.google.com/github/oceane0815/Pink_Tax/blob/main/ACSAPI_SEX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#!pip install --upgrade --force-reinstall pandas

import requests
import pandas as pd
import re

# Replace with your Census API key
#api_key = "c308bc01c73f163cf900e55ac8eb7ca87e533b0f"
api_key = "95cbdc8791f01d04f65891a38cb8090856b346e3"

# List of years to download
years = [2018, 2019, 2020, 2021, 2022]

##### Request the Sex data from ACS, Group B01001 ############################

# List to store data for each year
data_frames = []

for year in years:
    # Construct URL for each year
    url = f"https://api.census.gov/data/{year}/acs/acs5?get=group(B01001)&for=state:*&key={api_key}"

    # Request data from the API
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()

        # Convert to DataFrame
        df = pd.DataFrame(data[1:], columns=data[0])

        # Add a column for the year
        df['year'] = year

        # Append to list of dataframes
        data_frames.append(df)
    else:
        print(f"Failed to retrieve data for {year}")

# Combine all years into one DataFrame
combined_data = pd.concat(data_frames, ignore_index=True)

# Display or save the combined data
print(combined_data.head())

# List of common columns to keep
additional_columns = ["GEO_ID", "NAME", "state", "year"]

## KEEP THE REAL ESTIMATES
# Generate column names dynamically for B01001_001E to B01001_049E
b01001_columns = [f"B01001_{str(i).zfill(3)}E" for i in range(1, 50)]

# Combine the columns
columns_to_keep = b01001_columns + additional_columns

# Filter the DataFrame
filtered_df = combined_data[columns_to_keep]

# Save to a CSV file
filtered_df.to_csv("B01001_sex_by_age_2018_2022.csv", index=False)

########## REQUEST THE DETAILED TABLES ################################
# URL for the JSON metadata for 2023 ACS 1-Year variables
url_concept = "https://api.census.gov/data/2023/acs/acs1/variables.json"

# Request the data
response_concept = requests.get(url_concept)
variables_concept = response_concept.json()

# Extract variable information into a list of dictionaries
variable_info = [
    {
        "variable": var,
        "label": details.get("label", ""),
        "concept": details.get("concept", "")
    }
    for var, details in variables_concept['variables'].items()
]

# Convert to DataFrame for easy filtering and display
df_dictionary = pd.DataFrame(variable_info)

# Display the mapping of variable names to concepts
print(df_dictionary.head())

# Step 1: Melt df to a long format
df_long = filtered_df.melt(id_vars=["GEO_ID", "NAME", "state", "year"], var_name="variable", value_name="value")

# Step 2: Convert the 'value' column to numeric
df_long['value'] = pd.to_numeric(df_long['value'], errors='coerce') # Convert 'value' to numeric

# Step 2: Merge with df_variables to add concept and label columns
df_long = df_long.merge(df_dictionary[['variable', 'label', 'concept']], how="left", on="variable")
print(df_long.head())

#################### Convert "concept" and "label" to one "label" col #########################

# Define a function to process each row
def combine_concept_label(row):
    # Extract and process the concept for race/ethnicity
    concept = row['concept']
    processed_concept = re.sub(r"[\(\)]", "", concept).replace(" ", "_")

    # Extract and process the label for gender and age
    label = row['label']
    if "Male" in label or "Female" in label:
        # Determine gender
        gender = "Male" if "Male" in label else "Female"

        # Check if there is an age group
        age_group = label.split("!!")[-1].strip()  # Extract age group or final part

        # Handle different cases
        if age_group == "Total":
            result = f"{processed_concept}_{gender}_Total"
        else:
            # Specific age group case
            age_group = age_group.replace(" ", "_")  # Replace spaces with underscores
            result = f"{processed_concept}_{gender}_{age_group}"
    else:
        # General total without gender
        result = f"{processed_concept}_Total"

    return result

# Apply the function to each row
df_long['combined_label'] = df_long.apply(combine_concept_label, axis=1)

# Display the first few rows to verify the results
print(df_long[['variable', 'label', 'concept', 'combined_label']].head())

####################################################################################


# Step 3: Pivot to make each variable a separate column while keeping concept and label as columns
df_wide = df_long.pivot_table(
    index=["GEO_ID", "NAME", "state", "year", "concept", "label"],
    columns="variable",
    values="value",
    aggfunc="first"
)

# Flatten the MultiIndex columns, if any
df_wide.columns = [col for col in df_wide.columns]

# Reset index to make it a clean DataFrame
df_wide = df_wide.reset_index()

# Display the final DataFrame
print(df_wide.head())




# Step 3: Merge df_long with df_variables to add 'label' and 'concept' columns
df_merge = df_long.merge(df_dictionary[['variable', 'label', 'concept']], how="left", on="variable")

# Step 4: Reshape back to the original wide format with label and concept columns per variable
df_final = df_merge.pivot_table(index=["GEO_ID", "NAME", "state", "year"], columns="variable", values=["value", "label", "concept"])

# Flatten the MultiIndex columns for readability (Optional)
df_final.columns = [f"{col[1]}_{col[0]}" for col in df_final.columns]

# Reset index to keep it as a DataFrame, not a pivoted DataFrame
df_final = df_final.reset_index()

# Display the final DataFrame
print(df_final.head())

# Save to a CSV file
df_long.to_csv("B01001_sex_by_age_2018_2022_long.csv", index=False)




ConnectTimeout: HTTPSConnectionPool(host='api.census.gov', port=443): Max retries exceeded with url: /data/2018/acs/acs5?get=group(B01001)&for=state:*&key=95cbdc8791f01d04f65891a38cb8090856b346e3 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7ed30c517af0>, 'Connection to api.census.gov timed out. (connect timeout=None)'))